In [1]:
import pickle
import plotly.plotly as py
from plotly.graph_objs import *
import pandas as pd
import numpy as np
from ase.units import kB
import cufflinks as cf
import plotly
plotly.tools.set_credentials_file(username='alatimer', api_key='qVVsdblMUDBO0SgYDLSV')

In [2]:
def dEa2dGa(dEa,T):
    return dEa - 3.942e-4*T - 0.0289
def err_fun(T):
    return 4.7e-5*T+0.0572
def dEafun(T):
    if T<=425:
        return 0.55-0.21
    elif T>425:
        return 0.55
def namefun(T):
    if T<=425:
        return "Selectivity Limit ~ Aqueous"
    elif T>425:
        return "Selectivity Limit ~ Gas-phase"
def sel_fun(conv,T,dEa=None,dGa=None,error=None):
    if dGa == None:
        dGa = dEa2dGa(dEa,T)
    if error=='+':
        dGa+=err_fun(T)
    elif error=='-':
        dGa-=err_fun(T)
    k2_k1 = np.exp(dGa/kB/T)
    sel = (1-conv-(1-conv)**(k2_k1))/(conv*(k2_k1-1))*100
    return sel #in percent

In [34]:
df = pd.read_csv('exp.dat',delim_whitespace=True)
#remove unneeded columns
smalldf = df[['category','DOI','log_conv','sel','T','rxntype','single-site']] 
#Remove non-single site, mmo, sel=0
smalldf = smalldf[(smalldf['sel']!=0) & (smalldf['category'].apply(lambda x: 'MMO' not in x))]
smalldf= smalldf[smalldf['single-site']=='yes']

In [57]:
conv_vec = np.logspace(-8,-.01,num=1e2,base=10)
temp_range = np.arange(275,975,50)
clr='lightblue' #color of selectivity limit lines

data = [ dict( 
        line=dict(color=clr),
        visible = False,
        name = namefun(temp),
        x = np.log10(conv_vec), 
        y = sel_fun(conv_vec,temp,dEa=dEafun(temp) )) for temp in temp_range 
       ]
    
data[10]['visible'] = True

steps = []
for i,temp in enumerate(temp_range):
    step = dict(
        method = 'restyle',  
        label=temp,
        args = ['visible', [False] * len(data) ],
    )
    step['args'][1][i] = True # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [ dict(
    active = 0,
    currentvalue = {"prefix": "Temperature (K): "},
    pad = {"t": 50},
    steps = steps
    ) 
 ]

layout = dict( 
                #include link to our paper
                title='Interactive version of Figure 5 from: <br> <i> Direct Methane to Methanol: The Selectivity-Conversion Limit and Design Strategies </i> <br>  Click on data to go to publication',
                xaxis=dict(
                    title='log(Methane Conversion)',
                    ),
                yaxis=dict(
                    title='Methanol Selectivity (%)',
                    ),
                sliders = sliders, 
                legend = dict(x=0.1,y=0.1) ,
                autosize=False,
                width=700,
                height=500,
             )

#Add +1 sigma error
data+=[dict( 
        line=dict(color=clr,dash='dash'),
        visible = False,
        showlegend=False,
        x = np.log10(conv_vec), 
        y = sel_fun(conv_vec,temp,dEa=dEafun(temp)+err_fun(temp) )) for temp in temp_range 
       ]
#Add -1 sigma error
data+=[dict( 
        line=dict(color=clr,dash='dash'),
        visible = False,
        name = "+/-1 std",
        x = np.log10(conv_vec), 
        y = sel_fun(conv_vec,temp,dEa=dEafun(temp)-err_fun(temp) )) for temp in temp_range 
       ]

for cat in smalldf['category'].unique():
    for temp in temp_range:
        tempdf = smalldf[(smalldf['T']<temp+25) & (smalldf['T']>temp-25) & (smalldf['category']==cat)]
        my_text = []
        for i,row in tempdf.iterrows():
            #check if patent
            if row['DOI'].startswith('US') or row['DOI'].startswith('GB'):
                my_text.append(" <a href='http://www.google.com/patents/%s'> </a>"%(row['DOI']))
            else:
                my_text.append(" <a href='http://doi.org/%s'> </a>"%(row['DOI']))

        data_dict=dict(
            visible=False,
            x = tempdf['log_conv'],
            y = tempdf['sel'],
            #mode='markers',
            mode='markers+text',
            name=cat,
            text = my_text,
            textfont = {'color':['w' for i in tempdf]},
            #hoverinfo=tempdf['T'],
        )
        data.append(data_dict)
        

        

In [58]:
fig = dict( data = data, layout = layout )
py.iplot( fig, filename = 'Temperature slider' )